# **[추가]클래스 선언 방식**


## **1.환경준비**

### (1) 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary

### (2) 필요 함수 생성

* 딥러닝을 위한 데이터로더 만들기

In [ ]:
def make_DataSet(x_train, x_val, y_train, y_val, batch_size = 32) :

    # 데이터 텐서로 변환
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

    # TensorDataset 생성 : 텐서 데이터셋으로 합치기
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

    return train_loader, x_val_tensor, y_val_tensor

* 학습을 위한 함수

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)                  # 전체 데이터셋의 크기
    num_batches = len(dataloader)                   # 배치 크기
    tr_loss = 0
    model.train()                                   # 훈련 모드로 설정(드롭아웃 및 배치 정규화와 같은 계층을 훈련 모드로 변경)
    for batch, (X, y) in enumerate(dataloader):     # batch : 현재 배치 번호, (X, y) : 입력 데이터와 레이블
        X, y = X.to(device), y.to(device)           # X.to(device), y.to(device): 입력 데이터와 레이블을 지정된 장치(device, CPU 또는 GPU)로 이동

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        tr_loss += loss

        # Backpropagation
        loss.backward()             # 역전파를 통해 모델의 각 파라미터에 대한 손실의 기울기를 계산
        optimizer.step()            # 옵티마이저가 계산된 기울기를 사용하여 모델의 파라미터를 업데이트
        optimizer.zero_grad()       # 옵티마이저의 기울기 값 초기화. 기울기가 누적되는 것 방지

    tr_loss /= num_batches          # 모든 배치에서의 loss 평균

    return tr_loss.item()

* 검증을 위한 함수

In [ ]:
def evaluate(x_val_tensor, y_val_tensor, model, loss_fn, device):
    model.eval()                        # 모델을 평가 모드로 설정

    with torch.no_grad():               # 평가 과정에서 기울기를 계산하지 않도록 설정(메모리 사용을 줄이고 평가 속도를 높입니다.)
        x, y = x_val_tensor.to(device), y_val_tensor.to(device)
        pred = model(x)
        eval_loss = loss_fn(pred, y).item()    # 예측 값 pred와 실제 값 y 사이의 손실 계산

    return eval_loss, pred

* 학습곡선

In [ ]:
def dl_learning_curve(tr_loss_list, val_loss_list):

    epochs = list(range(1, len(tr_loss_list)+1))
    plt.plot(epochs, tr_loss_list, label='train_err', marker = '.')
    plt.plot(epochs, val_loss_list, label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (3) device 준비(cpu or gpu)

In [ ]:
# cpu 혹은 gpu 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
def to_numpy(tensor, device):
    if tensor.requires_grad:
        tensor = tensor.detach()
    return tensor.to('cpu').numpy() if device == 'cuda' else tensor.numpy()

## **2.모델링1 : 회귀**

### (1) 데이터 준비 : boston

* 데이터 다운로드

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv'
data1 = pd.read_csv(path)
data1.head()

* 데이터 분할

In [ ]:
target1 = 'medv'
x1 = data1.drop(target1, axis = 1)
y1 = data1.loc[:, target1]
x1_train, x1_val, y1_train, y1_val = train_test_split(x1, y1, test_size=.2, random_state = 20)

In [ ]:
x1_train.shape

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x1_train = scaler.fit_transform(x1_train)
x1_val = scaler.transform(x1_val)

* 텐서 데이터셋, 데이터 로더

In [ ]:
train_loader1, x1_val_ts, y1_val_ts = make_DataSet(x1_train, x1_val, y1_train, y1_val)

### (2) 기존방식

* 모델 선언

In [ ]:
n_features = x1_train.shape[1]

# 모델 구조 설계
model11 = nn.Sequential(
		nn.Linear(n_features, 3),
		nn.ReLU(),
		nn.Linear(3, 1)
).to(device)

# loss, optimizer
loss_fn = nn.MSELoss()
optimizer = Adam(model11.parameters(), lr=0.1)

# 모델 요약
summary(model11, input_size = (12,))

* 학습

In [ ]:
epochs = 50
tr_loss_list, val_loss_list = [], []

for t in range(epochs):
    tr_loss = train(train_loader1, model11, loss_fn, optimizer, device)
    val_loss, _ = evaluate(x1_val_ts, y1_val_ts, model11, loss_fn, device)

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 모델 평가

In [ ]:
loss, pred = evaluate(x1_val_ts, y1_val_ts, model11, loss_fn, device)

y1_val = to_numpy(y1_val_ts, device)
pred = to_numpy(pred, device)

mae = mean_absolute_error(y1_val, pred)
mape = mean_absolute_percentage_error(y1_val, pred)

print(f'MSE : {loss}')
print(f'MAE : {mae}')
print(f'MAPE : {mape}')

### (3) 클래스 방식

* 클래스 생성

In [ ]:
class CustomNN1(nn.Module):
    def __init__(self, n_features):
        super(CustomNN1, self).__init__()
        # 필요한 레이어 정의(초기화)
        self.fc1 = nn.Linear(n_features, 3)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(3, 1)

    def forward(self, x):
        # 모델 설계
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

* 모델 선언

In [ ]:
n_features = x1_train.shape[1]
model12 = CustomNN1(n_features).to(device)

loss_fn = nn.MSELoss()
optimizer = Adam(model12.parameters(), lr=0.1)

# 모델 요약
summary(model12, input_size = (12,))

* 학습

In [ ]:
epochs = 50
tr_loss_list, val_loss_list = [], []

for t in range(epochs):
    tr_loss = train(train_loader1, model12, loss_fn, optimizer, device)
    val_loss, _ = evaluate(x1_val_ts, y1_val_ts, model12, loss_fn, device)

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 모델 평가

In [ ]:
loss, pred = evaluate(x1_val_ts, y1_val_ts, model12, loss_fn, device)

y1_val = to_numpy(y1_val_ts, device)
pred = to_numpy(pred, device)

mae = mean_absolute_error(y1_val, pred)
mape = mean_absolute_percentage_error(y1_val, pred)

print(f'MSE : {loss}')
print(f'MAE : {mae}')
print(f'MAPE : {mape}')

## **3.모델링2 : 분류**

### (1) 데이터 셋

* 다운로드

In [ ]:
# Download training data from open datasets.
train_dataset = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),   # 픽셀값을 [0,1] 사이로 정규화하고 텐서로 변
)

# Download test data from open datasets.
test_dataset = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

* 데이터 로더

In [ ]:
batch_size = 64
train_loader2 = DataLoader(train_dataset, batch_size=batch_size)

* val, test 준비

In [ ]:
x2_val, x2_test = test_dataset.data[:5000], test_dataset.data[5000:]
y2_val, y2_test = test_dataset.targets[:5000], test_dataset.targets[5000:]

In [ ]:
x2_val = x2_val / 255
x2_test = x2_test/ 255

In [ ]:
x2_val = x2_val.view(5000, 1, 28, 28)
x2_test = x2_test.view(5000, 1, 28, 28)

print(x2_val.shape, x2_test.shape)

### (2) 기존방식

* 모델 선언

In [ ]:
n_feature = 28 * 28
n_class = 10

# 모델 구조 설계
model21 = nn.Sequential(nn.Flatten(),               # 이미지를 옆으로 펼치기(한 행에 데이터를 넣기)
                      nn.Linear(28*28, n_class)
        ).to(device)

print(model21)

* Loss function과 Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model21.parameters(), lr=0.0001)

* 학습

In [ ]:
epochs = 5
tr_loss_list, val_loss_list = [], []

for t in range(epochs):
    tr_loss = train(train_loader2, model21, loss_fn, optimizer, device)
    val_loss,_ = evaluate(x2_val, y2_val, model21, loss_fn, device)

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 학습 곡선

In [ ]:
dl_learning_curve(tr_loss_list, val_loss_list)

* 모델 평가

In [ ]:
_, pred = evaluate(x2_test, y2_test, model21, loss_fn, device)
pred = nn.functional.softmax(pred, dim=1)

In [ ]:
pred = to_numpy(pred, device)
pred = np.argmax(pred, axis = 1)

* confusion matrix

In [ ]:
cm = confusion_matrix(y2_test, pred)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = train_dataset.classes)
disp.plot()
plt.xticks(rotation=90)
plt.show()

* classification_report

In [ ]:
print(f'Accuracy : {accuracy_score(y2_test, pred)}')
print('-'*60)
print(classification_report(y2_test, pred))

### (3) 클래스 방식

* 클래스 생성

In [ ]:
class CustomNN2(nn.Module):
    def __init__(self, n_class):
        super(CustomNN2, self).__init__()
        # 필요한 레이어 정의(초기화)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(28*28, n_class)

    def forward(self, x):
        # 모델 설계
        x = self.flat(x)
        x = self.fc(x)
        return x

* 모델 선언

In [ ]:
n_class = 10
model22 = CustomNN2(n_class).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model22.parameters(), lr=0.0001)

# 모델 요약
summary(model22, input_size = (1, 28, 28))

* 학습

In [ ]:
epochs = 5
tr_loss_list, val_loss_list = [], []

for t in range(epochs):
    tr_loss = train(train_loader2, model22, loss_fn, optimizer, device)
    val_loss,_ = evaluate(x2_val, y2_val, model22, loss_fn, device)

    # 리스트에 loss 추가 --> learning curve 그리기 위해.
    tr_loss_list.append(tr_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch {t+1}, train loss : {tr_loss:4f}, val loss : {val_loss:4f}")

* 모델 평가

In [ ]:
_, pred = evaluate(x2_test, y2_test, model22, loss_fn, device)
pred = nn.functional.softmax(pred, dim=1)

In [ ]:
pred = to_numpy(pred, device)
pred = np.argmax(pred, axis = 1)

* confusion matrix

In [ ]:
cm = confusion_matrix(y2_test, pred)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = train_dataset.classes)
disp.plot()
plt.xticks(rotation=90)
plt.show()

* classification_report

In [ ]:
print(f'Accuracy : {accuracy_score(y2_test, pred)}')
print('-'*60)
print(classification_report(y2_test, pred))

## **4.실습**



### (1) 실습1 : boston 데이터

* 여러분은 다음의 구조를 클래스 선언 방식으로 모델링을 수행하고 성능평가를 해 봅시다.
    * 모델 구조
            Sequential(
              (0): Linear(in_features=12, out_features=10, bias=True)
              (1): ReLU()
              (2): Linear(in_features=10, out_features=5, bias=True)
              (3): ReLU()
              (4): Linear(in_features=5, out_features=1, bias=True)
            )
    * epochs(반복횟수) : 100
    * lr(learning rate, 학습률) : 0.01

* Loss function과 Optimizer

* 학습

* 학습 곡선

* 모델 평가 : MSE

### (2) 실습2 : mnist 데이터

다음의 기존 모델 선언 내용을 클래스 방식으로 바꿔보세요.

* 기존 모델 설계

In [ ]:
n_class = 10

# 모델 구조 설계
model4 = nn.Sequential(
    nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(8 * 7 * 7, 128),
    nn.ReLU(),
    nn.Linear(128, n_class)
).to(device)

# loss, optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model4.parameters(), lr=0.001)

# 모델 요약
summary(model4, input_size = (1,28,28))

* 클래스 생성

* 모델 선언

* 학습

* 모델 평가

* confusion matrix

* classification_report